<a href="https://colab.research.google.com/github/kkrusere/Market-Basket-Analysis-on-the-Online-Retail-Data/blob/main/MBA_Online-Retail_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <center> **Market Basket Analysis on Online Retail Data** 
<center><em>Providing a Retail Business with a strategy which helps improve their product sales, inventory management, and customer retention, which in turn would improve the profitability of the business.</em></center>
<br>
<center><img src="https://github.com/kkrusere/Market-Basket-Analysis-on-the-Online-Retail-Data/blob/main/Assets/MBA.jpg?raw=1" width=600/></center>

***Project Contributors:*** Kuzi Rusere<br>
**MBA streamlit App URL:** https://kkrusere-market-basket-analysis-on-the-online-re-mba-app-oi5iot.streamlitapp.com/

### **Project Introduction**

***Business Proposition:*** This project aims to provide a Retail Business with a strategy that helps improve their product sales, inventory management, and customer retention, which in turn would improve the profitability of the business. In the retail environment, profitability and the `bottom line` is at the focal point of any organization or business in this space. From product sales, through inventory management to customer acquisition and retention all this one way or the other affects the business' profits and net revenue. Transaction data from the POS (point of sale) systems for a retail business is a treasure trove of insights that can be used to better understand the products, customer purchase behavior, and sales together with the relationship and patterns in between. This project explores the different ways of deriving these insights, patterns, and relationships, and how these can be used in designing, developing, and implementing a strategy to improve the retail business' profits, revenue, and overall operations<br>
<br><br>

***Methodology:*** Data Mining, Analysis and Visualization of Retail Sales Data.<br>
1. Market Basket Analysis (MBA), which aims to find relationship and establishing pattens within the retail sales data. <br>
2. Customer Segmentation 
> * RFM (recency, frequency, monetary) Analysis
> * Cohort analysis
3. Product Recomendation (people who bought this also bought)

In addition, <br>
We create a `Streamlit` interactive data visualization tool that allows users interact with the data and analytics.

### **Data Collection**


In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud

import calendar
import datetime as dt

import re

In [ ]:
#we are going to import the data into a pandas dataframe 
def data_loading(path):
    """
    This function reads the Excel data file from UCI Machinelearning repository 
    The data is stored into a pandas dataframe that the fuction returns  
    """
    df = pd.read_excel(path)
    return df

data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"

df = data_loading(data_url)

**General Information About the Data** <br>
This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers. <br>

**Information about the Attributes/Columns in the Dataset** <br>

***InvoiceNo:*** Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. <br>

***StockCode:*** Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product. <br>

***Description:*** Product (item) name. Nominal. <br>

***Quantity:*** The quantities of each product (item) per transaction. Numeric.<br>

***InvoiceDate:*** Invice Date and time. Numeric, the day and time when each transaction was generated. <br>

***UnitPrice:*** Unit price. Numeric, Product price per unit in sterling.<br>

***CustomerID:*** Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer. <br>

***Country:*** Country name. Nominal, the name of the country where each customer resides. <br>

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


We are going to use the pandas '.shape' function/method to the total number of columns and rows of the dataframe. We can see that our dataframe contains 541909 rows and 8 columns

In [ ]:
df.shape

(541909, 8)

We'll use the pandas '.info()' function so see the general infomation (data types, null value count, etc.) about the data.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


From the above we can see that the Description and CustomerID columns have null values, and that the CustomerID has data type float64. 
> * We are going to convert the CustomerID datatype to 'string' and ***Consider*** the null values in this columns ***Guest Customer*** implying that they are customers that do not have Customer IDs.<br> <br>
> * For the null values in the Description column, we are also going to delete the rows, since we do not have any use of them in this particular project.

### **Data Cleaning and Preparation**

We are going to create a function that will encompass all the cleaning and manipulation/modifications that we are going to do to the data.

**What we are going to do to the data frame:**<br>
* Changing the CustomerID to dtype 'object' and replace the null values to 'Guest Customer'.<br><br>
* Remove the rows that have null values from the Description column. we are also going to remove rows that have a product description that has a length less than or equal to 6 characters, this is because the description is not actually products (well from our understanding). Below is a sample of some of these descriptions 
> ['wet?','Damaged','Missing','Discount','lost','MIA','SAMPLES','Display','mailout ','?','missing?','broken','lost??','CARRIAGE','mouldy','smashed']
* Cancelled orders will be removed, since as per the info about the attributes/columns of our dataset, for the 'InvoiceNo' the entries that contain a 'c' is a canceled orders.<br><br>
* We are going to remove the data entries from 2010 and then split the 'Invoice Date' column into just Date (without the time), Time, Month, Day, Week, Day of the Week, Hour, Time of Day (which is either Morning, Afternoon or Evening). We are going to need these to dissect the transactions at different periods.<br><br>
* We are also going to remove and consolidate duplicated entries in the product description that have the same `stockcode`
* A 'Sales Revenue' column will be created by multiplying the 'Quantity' and 'UnitPrice' columns. This will give the monetary value of each data entry.


In [ ]:
def data_cleaning_and_prep(df):
    """  
    This is the Data Cleaning and Preperation Function
    It takes in a dataframe, does the data cleaning and prep and returns dataframe
    """
    ######*****CustomerID column******###########
    #replacing the nan with 'Guest Customer' so that we can convert to int to remove the decimal point
    df["CustomerID"] = df["CustomerID"].fillna("Guest Customer")

    ######*****InvoiceDate column******###########
    df['Date'] = df['InvoiceDate'].dt.date
    df['Month'] = df['InvoiceDate'].dt.month
    df['Year'] = df['InvoiceDate'].dt.year
    #we change the number month to the actual name of the month 
    df['Month'] = df['Month'].apply(lambda x: calendar.month_name[int(x)])
    df['Week of the Year'] = df['InvoiceDate'].dt.isocalendar().week
    #we will add the day of the week to the dataframe 
    df['Day of Week'] = df['InvoiceDate'].dt.day_name()
    df['Time'] = df['InvoiceDate'].dt.time
    df["Hour"] = df['InvoiceDate'].dt.hour
    df['Time of Day']=pd.cut(df['InvoiceDate'].dt.hour,[0,6,12,18,23],labels=['Night','Morning','Afternoon','Evening'],include_lowest=True)
    #lastly here, we will drop the row from the year 2010
    index_names = df[df['Year'] == 2010].index
    df.drop(index_names, inplace = True)
    df.drop('Year', axis=1, inplace=True)

    ######*****InvoiceNo column******#############
    #now we drop the row with cancelled orders
    df['InvoiceNo'] = df['InvoiceNo'].dropna()

    #####*****Sales Revenue column******##########
    #we'll create a column called 'Sales Revenue' = 'Quantity' multiplied by 'UnitPrice'
    df['Sales Revenue'] = df['UnitPrice'] * df['Quantity']

    #####*****Country column******##########
    #we are going to drop the rows that have "Unspecified" country
    df.drop(df[df["Country"] == "Unspecified"].index, inplace = True)

    ###drop row were the quantity and sales revenue that is less than 0 and drop duplicated rows if any
    index_names = df[df['Quantity'] <= 0].index
    df.drop(index_names, inplace = True)
    index_names = df[df['Sales Revenue'] <= 0].index
    df.drop(index_names, inplace = True)
    df.drop_duplicates(inplace = True)

    ######*****Description column******###########
    #dropping the nan from the Description column
    df.dropna(subset = ['Description'], inplace= True)
    #removing the descriptions with lenth less than or equal to 8 characters
    df.drop(df[df.Description.str.len() <= 8].index, inplace=True)
    #we are going to drop this special case row with description '20713' at index '417090'
    df.drop(df.index[417090], inplace=True)
    #the description to lower case
    df.Description = df.Description.str.lower()
    #we are also going to remove the row that have description value "dotcom postage"
    index_names = df[ df['Description'] == "dotcom postage" ].index
    # drop these row indexes
    # from dataFrame
    df.Description = df.Description.str.upper()
    df["Description"] = df["Description"].str.strip()
    df.drop(index_names, inplace = True)
    #getting rid of the duplicated product discription
    the_dict = dict()
    df.reset_index(drop=True, inplace=True)
    for i, code in enumerate(df["StockCode"]):
        if code not in the_dict:
            the_dict[code] = list()
        if df["Description"][i] not in the_dict[code]:
            the_dict[code].append(df["Description"][i])

    dict_to_look_at = dict()
    for key in the_dict:
        if len(the_dict[key]) > 1:
            dict_to_look_at[key] = the_dict[key]

    key_list = list(dict_to_look_at.keys())
    for key_l in key_list:
        x = dict_to_look_at[key_l]
        max_len = 0
        index = 0
        for i in range(len(x)):
            if len((x[i]).split()) >= max_len:
                max_len = len((x[i]).split())
                index = i
        dict_to_look_at[key_l] = x[index]

    for key in dict_to_look_at:
        df.loc[df["StockCode"] == key, "Description"] = dict_to_look_at[key]

    return df

df = data_cleaning_and_prep(df)

After cleaning the data we are left with 481313 and now have 16 columns. 

In [ ]:
df.shape

(481313, 16)

In [ ]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Date,Month,Week of the Year,Day of Week,Time,Hour,Time of Day,Sales Revenue
0,539993,22386,JUMBO BAG PINK POLKADOT,10,2011-01-04 10:00:00,1.95,13313.0,United Kingdom,2011-01-04,January,1,Tuesday,10:00:00,10,Morning,19.5
1,539993,21499,BLUE POLKADOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom,2011-01-04,January,1,Tuesday,10:00:00,10,Morning,10.5
2,539993,21498,RED RETROSPOT WRAP,25,2011-01-04 10:00:00,0.42,13313.0,United Kingdom,2011-01-04,January,1,Tuesday,10:00:00,10,Morning,10.5
3,539993,22379,RECYCLING BAG RETROSPOT,5,2011-01-04 10:00:00,2.10,13313.0,United Kingdom,2011-01-04,January,1,Tuesday,10:00:00,10,Morning,10.5
4,539993,20718,RED RETROSPOT SHOPPER BAG,10,2011-01-04 10:00:00,1.25,13313.0,United Kingdom,2011-01-04,January,1,Tuesday,10:00:00,10,Morning,12.5


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481313 entries, 0 to 481312
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   InvoiceNo         481313 non-null  object        
 1   StockCode         481313 non-null  object        
 2   Description       481313 non-null  object        
 3   Quantity          481313 non-null  int64         
 4   InvoiceDate       481313 non-null  datetime64[ns]
 5   UnitPrice         481313 non-null  float64       
 6   CustomerID        481313 non-null  object        
 7   Country           481313 non-null  object        
 8   Date              481313 non-null  object        
 9   Month             481313 non-null  object        
 10  Week of the Year  481313 non-null  UInt32        
 11  Day of Week       481313 non-null  object        
 12  Time              481313 non-null  object        
 13  Hour              481313 non-null  int64         
 14  Time

In [ ]:
df.describe()

,Quantity,UnitPrice,Week of the Year,Hour,Sales Revenue
count,481313.00000,481313.000000,481313.000000,481313.000000,481313.000000
mean,10.80504,3.252614,29.164677,13.066221,19.693793
std,162.79915,16.522079,14.743237,2.448566,280.999991
min,1.00000,0.001000,1.000000,6.000000,0.001000
25%,1.00000,1.250000,16.000000,11.000000,3.900000
50%,4.00000,2.080000,31.000000,13.000000,9.950000
75%,12.00000,4.130000,43.000000,15.000000,17.700000
max,80995.00000,11062.060000,49.000000,20.000000,168469.600000


At this point, we load the clean data using the function above. We are also going to save the data to our AWS RDS MySQL database for later use in the Streamlit application of this project.

In [ ]:
# import mysql.connector as connection
# from sqlalchemy import create_engine
# import config

# host=config.host
# user=config.user
# password=config.password
# port = config.port

# conn = connection.connect(
#   host=host,
#   user=user,
#   password=password,
#   port = port,
# )
# mycursor = conn.cursor()

# database = config.database

# mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {database}")

# mycursor.execute("SHOW DATABASES")

# # create sqlalchemy engine
# engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")
# df.to_sql('MBA_Online-Retail_Data', con = engine, if_exists = 'append', index=False)

481313

### **Exploratory Data Analysis *(EDA)***

* Exploratory data analysis is an approach/practice of analyzing data sets to summarize their main characteristics, often using statistical graphics and other ***data visualization***. It is a critical process of performing initial ***investigations to discover*** patterns, detect outliers and anomalies, and gain some new, hidden, insights into the data.

* Investigating questions like the total volume of purchases per month, week, day of the week, time of the day right to the hour. We will look at customers more later when we get into the ***Recency, Frequency and Monetary Analysis (RFM)***, ***Cohort analysis***, and the ***Customer Profitability Analysis (CPA)*** sections of the project.

We are going to create a helper fuction that will group the Quantity and Sales Revenue with respect to either the 'CustomerID', 'Country', 'Date', 'Month','Week of the Year', 'Day of Week', 'Hour', or 'Time of Day'.

In [ ]:
def group_Quantity_and_SalesRevenue(df,string):
    """ 
    This function inputs the main data frame and feature name 
    The feature name is the column name that you want to group the Quantity and Sales Revenue
    """

    df = df[[f'{string}','Quantity','Sales Revenue']].groupby([f'{string}']).sum().sort_values(by= 'Sales Revenue', ascending = False).reset_index()

    return df

In [ ]:
Country_Data = df.groupby("Country")["InvoiceNo"].nunique().sort_values(ascending = False).reset_index().head(10)
fig = px.bar(Country_Data, x= "InvoiceNo", y='Country', title= "Top 10 Number of orders per country with the UK")
#fig.show(renderer='png', height=700, width=1000)
fig.show( height=700, width=1000)

UK has more number of orders witk 16k Invoice numbers 

In [ ]:
Country_Data = df[df['Country'] != "United Kingdom"].groupby("Country")["InvoiceNo"].nunique().sort_values(ascending = False).reset_index().head(10)
fig = px.bar(Country_Data, x= "InvoiceNo", y='Country', title= "Top 10 Number of orders per country without the UK")
#fig.show(renderer='png', height=700, width=1000)
fig.show( height=700, width=1000)

### **Market Basket Analysis**

### **Customer Segmentation**
* RFM (recency, frequency, monetary) Analysis
* Customer Profitability Analysis (CPA)
* Cohort analysis

### **Product recomendation (people who bought this also bought)**